
# 使用Gradio构建交互式AI应用程序

本实验指导您使用Gradio创建交互式AI应用程序，重点是构建具有流式响应的可定制聊天机器人。

Gradio是一个Python前端库，使构建聊天机器人体验变得非常容易。

## 先决条件

开始之前，请确保您拥有必要的包：

In [ ]:
# Install required packages
!pip install gradio openai python-dotenv

## 1. 设置您的环境

In [ ]:
import os
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables (if using .env file)
load_dotenv()

# IMPORTANT: Set your API key directly (for testing only)
# NOTE: For security reasons, you should use a .env file instead in production
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
# Now initialize the OpenAI client

# Verify API key
if os.environ.get("OPENAI_API_KEY"):
    print("OpenAI API Key loaded successfully")
else:
    print("ERROR: OpenAI API Key not found!")

## 2. 您的第一个Gradio界面

让我们创建一个简单的Gradio界面：

In [ ]:
# We will define a simple function called echo, all it does is it will accept a message 
# and simply return it back!
def echo(message):
    return f"You said: {message}"

# Create a basic Gradio interface. gr.interface() is a function that creates a web-based UI 
# for your function, basically what that means is that it will create a web page where you 
# can enter text and see the output. That interface will be hosted on a local server, and 
# you can access it via your web browser.
# The interface accepts the function to be called (echo in this case), the input type 
# (textbox), and the output type (also textbox). The title and description are optional, 
# but they help to provide context for the user. Flagging mode is set to "never" to 
# disable the flagging feature to make it look better.
# When you run this function, it will spin up an interface with input and output boxes, 
# and a button to submit the input. When you submit the input, it will call the echo 
# function with the input text and display the output in the output box.
demo = gr.Interface(
    fn=echo,
    inputs="textbox",
    outputs="textbox",
    title="Echo Bot",
    description="A simple bot that echoes what you type",
    flagging_mode="never"
)

# Launch the interface
demo.launch()

## 3. 构建基本聊天机器人

让我们使用OpenAI创建一个简单的聊天机器人：

In [ ]:
# Function to get response from OpenAI. We already went through that before. 
def get_response(message):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": message}
        ]
    )
    return response.choices[0].message.content

# We will create the same exact gradio interface as before, but we will use get_response 
# function instead of echo! The get_response function will call the Open AI with the 
# message and return the response. The response will be displayed in the output box.
# Basically, the user will enter a message in the input box, and when they click the 
# submit button, it will call the get_response function with the input text and display 
# the output in the output box.

chatbot = gr.Interface(
    fn=get_response,
    inputs=gr.Textbox(placeholder="Ask me anything..."),
    outputs="text",
    title="Simple AI Chatbot",
    description="Chat with an AI assistant powered by OpenAI",
    flagging_mode="never"
)

# Launch the chatbot
chatbot.launch()

## 4. 创建流式聊天机器人

In [ ]:
# Function to stream responses from OpenAI
def stream_response(message):
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": message}
        ],
        stream=True
    )
    
    response = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            response += chunk.choices[0].delta.content
            # yield is a python keyword that allows you to return a value from a function 
            # without terminating the function. This means that the function can be called 
            # again later, and it will continue from where it left off. This is useful 
            # for streaming responses, as it allows you to return partial results as they 
            # are generated. So this is instead of return, which would terminate the function.
            yield response 

# Create a streaming chatbot interface
streaming_chatbot = gr.Interface(
    fn=stream_response,
    inputs="textbox",
    outputs="text",
    title="Streaming AI Chatbot",
    description="Chat with an AI assistant that streams responses in real-time",
    flagging_mode="never"
)

# Launch the streaming chatbot
streaming_chatbot.launch()

## 5. 构建带记忆的聊天机器人

现在让我们创建一个更高级的聊天机器人，它能记住对话历史。

我们之前已经经历过这个过程，我们只是将其添加到Gradio界面中。

In [ ]:
# Define a function called chat, which accepts the user message and conversation history
def chat(message, history):
    # Initialize messages with system prompt
    messages = [{"role": "system", "content": "You are a helpful assistant."}]
    
    # We are looping through the history of messages (initially just an empty list). 
    # Entry is a single message in the history
    for entry in history:
        # Check if entry is in list format [user_msg, assistant_msg]
        if isinstance(entry, list) and len(entry) == 2:
            user_msg, assistant_msg = entry
            # Add user message
            messages.append({"role": "user", "content": user_msg})
            # Add assistant message if it exists
            if assistant_msg:
                messages.append({"role": "assistant", "content": assistant_msg})
        # If history is already in OpenAI format with role and content
        elif isinstance(entry, dict) and "role" in entry and "content" in entry:
            messages.append(entry)
    
    # Add the new user message to the conversation
    messages.append({"role": "user", "content": message})
    
    # Get streaming response from OpenAI
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        stream=True
    )
    
    # Return the response chunk by chunk as it arrives
    response = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            response += chunk.choices[0].delta.content
            yield response

# Create and launch the chatbot interface
memory_chatbot = gr.ChatInterface(
    fn=chat,
    title="AI Chatbot with Memory",
    description="Chat with an AI assistant that remembers your conversation history.",
    examples=["Tell me about machine learning", "How do neural networks work?"],
    flagging_mode="never"
)

memory_chatbot.launch()

## 6. 餐厅菜单生成器

In [ ]:
# Function to generate a restaurant menu
def generate_menu(restaurant_name, cuisine_type, special_requirements="None"):
    prompt = f"""
    Create a restaurant menu for "{restaurant_name}", a {cuisine_type} restaurant.
    Special requirements: {special_requirements}
    
    Include:
    - 3 appetizers
    - 4 main courses
    - 2 desserts
    
    For each item, include a name, brief description, and price.
    
    Format your response in markdown with appropriate headers, styling, and sections.
    Add a brief introduction about the restaurant at the top.
    """
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an expert restaurant consultant who creates beautiful, well-formatted menus."},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response.choices[0].message.content

# Create a menu generator interface
menu_generator = gr.Interface(
    fn=generate_menu,
    inputs=[
        gr.Textbox(label="Restaurant Name"),
        gr.Textbox(label="Cuisine Type (e.g., Italian, Japanese)"),
        gr.Textbox(label="Special Requirements (Optional)", placeholder="e.g., vegetarian options")
    ],
    outputs=gr.Markdown(label="Generated Menu"),
    title="Restaurant Menu Generator",
    description="Create a professional restaurant menu with AI",
    flagging_mode="never"
)

# Launch the menu generator
menu_generator.launch()

## 7. 进一步学习资源

- [Gradio文档](https://www.gradio.app/docs/)
- [OpenAI API文档](https://platform.openai.com/docs/api-reference)